In [ ]:
import json

import matplotlib.pyplot as plt
import numpy as np

In [ ]:
graph_dict = json.load(open("graph_dict.json"))

In [ ]:
from simulator import Simulator

In [ ]:
sim = Simulator(graph_dict=graph_dict)
sim.run_conf()

In [ ]:
data=np.load("./data/simulation_8.npz")
time_series = data['t']
state_var = data['y']

In [ ]:
# Code to show graph

aggregated = np.sum(state_var, axis=2)

fig, ax = plt.subplots()
ax.plot(time_series, aggregated[:, sim.c_idx["S"]], "g")
ax.plot(time_series, aggregated[:, sim.c_idx["E"]], "r")
ax.plot(time_series, aggregated[:, sim.c_idx["I"]], "b")
ax.plot(time_series, aggregated[:, sim.c_idx["R"]], "purple")
ax.legend(["Susceptibles", "Exposed", "Infected", "Recovered"], loc="best") 
plt.show()

In [ ]:
# Code to show graph

fig, ax = plt.subplots()
ax.plot(time_series, state_var[:, sim.c_idx["S"], 0], "g")
ax.plot(time_series, state_var[:, sim.c_idx["S"], 1], "r")
ax.plot(time_series, state_var[:, sim.c_idx["S"], 2], "b")
ax.legend(["Age group 0-14", "Age group 15-59", "Age group 60+"], loc="best") 
plt.show()